In [236]:
from google.colab import drive
from google.colab import files
import re
import numpy as np
import string
import random
from math import *

In [237]:
files.upload()

Saving The Adventures of Sherlock Holmes.txt to The Adventures of Sherlock Holmes.txt


{'The Adventures of Sherlock Holmes.txt': b'\n\n\n\n\n\n<!DOCTYPE html>\n<html lang="en" >\n  <head>\n    <meta charset="utf-8">\n  <link rel="dns-prefetch" href="https://github.githubassets.com">\n  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">\n  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">\n  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">\n\n\n\n  <link crossorigin="anonymous" media="all" integrity="sha512-oDd/zRd70xTilBt9+3Pc1+2Z8hlcfG7eQeFjLMayonKEml+i6KgFoG1ibLQHA1wM55KTOj+FSPxuDgLYfJ1exw==" rel="stylesheet" href="https://github.githubassets.com/assets/frameworks-a0377fcd177bd314e2941b7dfb73dcd7.css" />\n  <link crossorigin="anonymous" media="all" integrity="sha512-OQfNnz+Vk23B3pL5IJn/1raKKLNAbQdlzxU0wk+8oPT9j1rsCDwnmUdy2lyuDtbpjKpVIEGtq3gSvvSp8nlUUQ==" rel="stylesheet" href="https://github.githubassets.com/assets/site-3907cd9f3f95936dc1de92f92099ffd6.css" />\n    <link crossorigin="anonymous" media="

In [219]:
drive.mount('content')

Mounted at content


In [220]:
def prepare_text(text):
  text = re.sub('[^a-zA-Z]', ' ', text)
  prepared_text = text.lower()
  return prepared_text

In [221]:
def encode_message(message):
  letters = string.ascii_lowercase
  letters_map = list(letters)
  random.shuffle(letters_map)
  letters_map = ''.join(letters_map)

  map_dict = dict()
  encoded_message = ''

  for index in range(len(letters)):
    map_dict[letters[index]] = letters_map[index]

  prepared_message = prepare_text(message)

  for char in prepared_message:
    if ord(char) == 32:
      encoded_message += ' '
    else:
      encoded_message += map_dict[char]

  return encoded_message

In [222]:
def decode_message(encoded_message):
  population = create_population(100)
  best_map = None
  best_prob = float('-inf')
  probs = list()
  chromosomes = list()
  decoded_by_population = ''

  for iters in range(700000):
    for chromosome in population:
      for char in encoded_message:
        if ord(char) == 32:
          decoded_by_population += ' '
        else:
          decoded_by_population += chromosome[char]

      prob = calculate_probs_text(decoded_by_population)

      if prob > best_prob:
        best_prob = prob
        best_map = chromosome

      probs.append(prob)
      chromosomes.append(chromosome)
      decoded_by_population = ''

    probs = np.array(probs)
    probs_sum = probs.sum()
    probs /= probs_sum
    alive_population = random.choices(range(len(chromosomes)), weights=probs, k=50)
    population_should_dead = list(range(len(chromosomes)))

    for i in list(set(alive_population)):
      population_should_dead.remove(i)

    for index_alive in alive_population:
      index_dead = random.choice(population_should_dead)
      population[index_dead] = mutation(population[index_alive])

    probs = list()
    chromosomes = list()

  return best_map, best_prob

In [223]:
def mutation(chromosome):
  key1 = chr(random.randint(ord('a'), ord('z')))
  key2 = chr(random.randint(ord('a'), ord('z')))

  chromosome[key1], chromosome[key2] = chromosome[key2], chromosome[key1]

  return chromosome

In [224]:
def create_population(population_count):
  chromosome = None
  population = list()
  population_map = dict()
  letters = string.ascii_lowercase

  for _ in range(population_count):
    chromosome = list(string.ascii_lowercase)
    random.shuffle(chromosome)

    for index in range(len(chromosome)):
      population_map[letters[index]] = chromosome[index]

    population.append(population_map)
    population_map = dict()

  return population

In [225]:
def count_chars():
  for char_index in range(len(text)):
    char0 = text[char_index]
    char0_num = ord(char0)
    if char0_num is not 32:
      char0_num -= 97
      prob_vector[char0_num] += 1
      char1 = text[char_index + 1]
      char1_num = ord(char1)
      if char1_num is not 32:
        char1_num -= 97
        prob_matrix[char0_num, char1_num] += 1
      else:
        continue
    else:
      continue

In [226]:
def calculate_probs():
  for row in range(prob_matrix.shape[0]):
    row_sum = prob_matrix[row].sum()
    for column in range(prob_matrix.shape[1]):
      prob_matrix[row, column] /= row_sum

  vector_sum = prob_vector.sum()
  for column in range(prob_vector.shape[0]):
    prob_vector[column] /= vector_sum

In [227]:
def calculate_probs_text(text):
  words = text.split(' ')
  for word in words:
    char0 = word[0]
    char0_num = (ord(char0) - 97)
    p = (prob_vector[char0_num])
    for char1 in word[1:]:
      char1_num = (ord(char1) - 97)
      p *= (prob_matrix[char0_num, char1_num])
      char0 = char1
      char0_num = (ord(char0) - 97)

  return p

In [238]:
file_read = open('moby10b.txt', 'r')
dataset = file_read.read()
file_read.close()
dataset += '\n'
file_read = open('book-war-and-peace.txt', 'r')
dataset = file_read.read()
file_read.close()
dataset += '\n'
file_read = open('The Adventures of Sherlock Holmes.txt', 'r')
dataset = file_read.read()
file_read.close()

In [239]:
len(dataset)

3136424

In [240]:
text = prepare_text(text=dataset)

In [241]:
prob_matrix = np.zeros((26, 26))
prob_vector = np.zeros((26))

In [242]:
count_chars()

In [243]:
calculate_probs()

In [244]:
original_message = 'god'

In [245]:
encoded_message = encode_message(message=original_message)

In [246]:
encoded_message

'rvn'

In [247]:
decoded_message = decode_message(encoded_message)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in true_divide


In [248]:
best_map = decoded_message[0]
decoded_message = ''

for char in encoded_message:
  if ord(char) == 32:
    decoded_message += ' '
  else:
    decoded_message += best_map[char]

In [249]:
decoded_message

'fgk'